# **London Underground ETL**

## Objectives

* Clean and merge Footfall and Station Coordinates datasets

## Inputs

* To run this notebook the StationFootfall and Stations datasets are required 

## Outputs

* ug_clean.csv will be created as our cleaned dataset





---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Extract

* Retrieving datasets and checking values

Importing necessary libraries

In [ ]:
import pandas as pd

Parsing CSV files into DataFrames

In [ ]:
Footfall = pd.read_csv(r'c:\Users\jackr\OneDrive\Desktop\my_projects\Project-1\Dataset\Dirty\StationFootfall_2024_2025 .csv')
Station_Coords = pd.read_csv(r'c:\Users\jackr\OneDrive\Desktop\my_projects\Project-1\Dataset\Dirty\Stations_20180921.csv')

Checking variable types 

In [ ]:
Footfall.info()
Station_Coords.info()

^ The 'x' and 'y' values are floats, this is ideal for us as it's necessary in creating map scatter plots later on

Displaying head of each DataFrame

In [ ]:
Footfall.head()

In [ ]:
Station_Coords.head()

---

# Transform

* Merging and Cleaning

Filtering non-underground stations

In [ ]:
Station_Coords = Station_Coords.query("NETWORK == 'London Underground'")

# Checking Networks have been filtered correctly          
Station_Coords.query("NETWORK == ['London Overground', 'Tramlink', 'DLR', 'TfL Rail']")
                            

^ No rows appear when querying our filtered stations, they are no longer in the DataFrame

Checking for duplicate values

In [ ]:
Footfall.duplicated().sum()
Station_Coords.duplicated().sum()

Cleaning whitespace in columns and values

In [ ]:
Footfall.columns = Footfall.columns.str.strip()
Station_Coords.columns = Station_Coords.columns.str.strip()
Footfall['Station'] = Footfall['Station'].str.strip()
Station_Coords['NAME'] = Station_Coords['NAME'].str.strip()


Merging Underground datasets

In [ ]:
merged_ug = pd.merge(
    Footfall,
    Station_Coords[['NAME', 'LINES', 'NETWORK', 'Zone', 'x', 'y']],
    left_on='Station',
    right_on='NAME',
    how='left'
)

merged_ug.head(10)

Dropping duplicate name column

In [ ]:
merged_ug = merged_ug.drop(columns=['NAME'], errors='ignore')
merged_ug.head()

Removing all rows with NaN values, these are non-underground stations

In [ ]:
merged_ug = merged_ug.dropna()
merged_ug.head()

Establishing new index

In [ ]:
merged_ug = merged_ug.set_index('Station')
merged_ug.head()

Changing Zone values to categorical variables

In [ ]:
merged_ug['Zone'] = merged_ug['Zone'].astype(int).astype('category')
merged_ug.head()

Changing TravelDate to correct datetime format

In [ ]:
merged_ug['TravelDate'] = pd.to_datetime(merged_ug['TravelDate'], format='%Y%m%d', errors='coerce')
merged_ug.head()

Checking if column types are as expected

In [ ]:
merged_ug.dtypes

# Load
* Creating a file for our cleaned data

Saves merged_ug as a CSV file into the Cleaned folder

In [ ]:
merged_ug.to_csv(r'Dataset\Cleaned\ug_clean.csv')

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Conclusions and Next Steps

* We retrieved the data we needed from the TfL (Transport for London) open data website
* Both Datasets were parsed into DataFrames
* The DataFrames were merged and cleaned according to the specifications we require
* The merged DataFrame was saved into the Cleaned folder

Our next steps should be thinking about how we approach the data visualisation stage. For our simpler stage we will use Matplotlib to create basic, easy to read plots such as line plots to show trends over time or pie charts to show proprtionality. 

Following on from this we will up the complexity slightly using Seaborn, with Seaborn the plots will be more visually appealing using color to show other variables. Additionally we can explore more complex plots like violin plots or heat maps.

Finally we can use plotly for interactive plots with hover data features as well as our crown jewel the scatter map plot.
